In [6]:
import requests

# URL for the login form
login_url = 'https://api.opencaselist.com/v1/login'

# Login credentials
login_data = {
    'username': 'ashleychen@live.com',
    'password': 's#170506'
}

# Start a session
session = requests.Session()

# Perform the login
response = session.post(login_url, json=login_data)

# Check if login was successful
if response.ok:
    print("Login successful!")
else:
    print("Login failed. Status code:", response.status_code)

# Extract cookies from the requests session
cookies = session.cookies.get_dict()
caselist = 'ndtceda23'


Login successful!


In [5]:
# connect to azure sql db
import pyodbc
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_SQL_CONNECTIONSTRING = os.environ['AZURE_SQL_CONNECTIONSTRING']

# Connect to the Azure SQL database
conn = pyodbc.connect(AZURE_SQL_CONNECTIONSTRING)



Connected to Azure SQL database


In [7]:
# populate teams
# extract data
schools_url = 'https://api.opencaselist.com/v1/caselists/{caselist}/schools'
schools = session.get(schools_url.format(caselist=caselist)).json()
cursor = conn.cursor()

teams_url = 'https://api.opencaselist.com/v1/caselists/{caselist}/schools/{school}/teams'
for school in schools:
    # fetch teams for each school
    teams = session.get(teams_url.format(caselist=caselist, school=school['name'])).json()
    
    # insert teams into db
    for team in teams:
        team_id = team['team_id']
        team_school = school['display_name']
        team_code4 = team['name']
        if len(team_code4) != 4:
            continue
        team_code2 = team_code4[0] + team_code4[2]
        team_code2_alt = team_code4[2] + team_code4[0]
        print(team_id, team_school, team_code4, team_code2, team_code2_alt)
        # cursor.execute("insert into teams (team_id, team_school, team_code4, team_code2, team_code2_alt) values (%s, %s, %s, %s, %s)", (team_id, team_school, team_code4, team_code2, team_code2_alt))
        # insert if not exist
        cursor.execute("if not exists (select * from teams where team_id = ?) insert into teams (team_id, team_school, team_code4, team_code2, team_code2_alt) values (?, ?, ?, ?, ?)", (team_id, team_id, team_school, team_code4, team_code2, team_code2_alt))
        
# close cursor
cursor.close()
# commit changes
conn.commit()

89093 Army CoCo CC CC
89095 Army McSh MS SM
89090 Army McTo MT TM
93168 Army OpSh OS SO
89091 Army PePe PP PP
90868 Army SoSt SS SS
89092 Army WiKa WK KW
91309 Baylor CaPe CP PC
88999 Baylor HuDa HD DH
89797 Baylor KeCh KC CK
89184 Baylor KePe KP PK
94841 Baylor LaHu LH HL
90551 Baylor LoPe LP PL
89183 Baylor LoSt LS SL
88780 Binghamton CoSa CS SC
88964 Binghamton KiTr KT TK
88965 Binghamton PiMa PM MP
88966 Binghamton T Ch TC CT
90460 Boston College HuCh HC CH
90461 Boston College McWa MW WM
90462 Boston College NoNo NN NN
94365 Cal State Fullerton AyDe AD DA
90822 Cal State Fullerton AyMa AM MA
88306 Cal State Fullerton DeVa DV VD
86878 Cal State Fullerton LoWi LW WL
88307 Cal State Fullerton MaPe MP PM
89424 Cal State Fullerton Pepe Pp pP
89237 Central Oklahoma GaSm GS SG
92459 Central Oklahoma SmWa SW WS
95259 Cherokee Trail VoCh VC CV
89446 Cornell AhBe AB BA
90426 Cornell AhDy AD DA
89453 Cornell BaCa BC CB
89455 Cornell BaKo BK KB
92934 Cornell CaTh CT TC
89456 Cornell ChMa CM M

In [9]:
import re

tournaments = set(["Northwestern", "JW Patterson", "Binghamton", "UCO", "Gotham", "MAC", "Run for the Roses", "Harvard", "Wake", "Kathryn", "Kansas State", "Kansas", "Mukai", "Rutgers RR", "Dartmouth RR", "Texas", "ADA", "CEDA", "NDT", "Houston"])

bad_to_good = {
    "J.W Patterson": "JW Patterson",
    "J.W. Patterson": "JW Patterson",
    "Kentucky Round Robin": "Run for the Roses",
    "Kentucky RR": "Run for the Roses",
    "Owen L Coon": "Northwestern",
    "Owen L Coon Memorial Debates": "Northwestern",
    "NUSO": "Northwestern",
    "nu": "Northwestern",
    "NU": "Northwestern",
    "Sunflower Swing Part 1": "Kansas",
    "Sunflower Swing pt1": "Kansas",
    "Sunflower Swing Part 1 hosted by KU": "Kansas",
    "Sunflower Swing Part 2": "Kansas State",
    "Sunflower Swing pt2": "Kansas State",
    "Shirley": "Wake",
    "Dartmouth": "Dartmouth RR",
    "Rutgers": "Rutgers RR",
    "ADA": "ADA",
    "CEDA": "CEDA",
    "NDT": "NDT",
    "National Debate Tournament": "NDT",
    "Northwestern": "Northwestern",
}

def clean_tournament_name(name):
    """Remove digits, symbols, and excess whitespace from a tournament name before the first alphabetic letter."""
    # Find the first alphabetic character and split the string at that point
    match = re.search(r'[a-zA-Z]', name)
    if not match:
        return name  # Return the original name if no alphabetic character is found
    
    split_index = match.start()
    prefix = name[:split_index]
    suffix = name[split_index:]
    
    # Clean up the prefix by removing non-alphabetic characters and excess whitespace
    cleaned_prefix = re.sub(r'[^a-zA-Z\s]', '', prefix)
    cleaned_prefix = re.sub(r'\s+', ' ', cleaned_prefix).strip()
    
    # Concatenate the cleaned prefix with the original suffix
    clean_name = cleaned_prefix + suffix
    
    if clean_name in tournaments:
        return clean_name
    
    if clean_name in bad_to_good:
        return bad_to_good[clean_name]
    
    else:
        for tournament in tournaments:
            if tournament.lower() in clean_name.lower():
                return tournament
    
    return clean_name

In [14]:
# populate rounds
rounds_url = 'https://api.opencaselist.com/v1/caselists/{caselist}/schools/{school}/teams/{team}/rounds'
cursor = conn.cursor()
cursor.execute("select team_id, team_code4, team_school, team_code2, team_code2_alt from teams")
teams = cursor.fetchall()
cursor.close()

# params: round_id, team_id, side [A, N], tournament, round, opponent, opensource
for team in teams:
    team_id = team[0]
    team_code = team[1].split(' ')[-1]
    team_school = team[2]

    response = session.get(rounds_url.format(caselist=caselist, school=team_school, team=team_code))
    rounds = response.json()
    for rnd in rounds:
        print(team_school, team_code, rnd['tournament'], rnd['round'], rnd['side'], rnd['opponent'], rnd['opensource'])
        side = "aff" if rnd['side'] == 'A' else "neg"
        tournament = clean_tournament_name(rnd['tournament'])
        print("tourn", tournament)

        # Check if the round is open source
        if rnd.get('opensource'):
            if not rnd['opponent']:
                continue
            opponent = rnd['opponent'].split(' ')
            opp_school = ' '.join(opponent[:-1])
            opp_code = opponent[-1]

            # Find opponent team_id
            cursor = conn.cursor()
            cursor.execute(
                "SELECT team_id FROM teams WHERE team_school = ? AND (team_code4 = ? OR team_code2 = ? OR team_code2_alt = ?)",
                (opp_school, opp_code, opp_code, opp_code)
            )
            opp_team_id = cursor.fetchall() # There may be multiple teams with the same code
            # Use the first team_id if there are multiple matches
            opp_team_id = opp_team_id[0] if opp_team_id else None
            # Remove tuple wrapper if there is only one team_id
            opp_team_id = opp_team_id[0] if opp_team_id else None

            opp_side = "aff" if side == "neg" else "neg"
            print("opponent", opp_school, opp_code, opp_team_id)

            # Check if the round already exists
            cursor.execute(
                f"SELECT round_id FROM caselist WHERE tournament_name = ? AND round_name = ? AND {opp_side}_team = ?",
                (tournament, rnd['round'], opp_team_id)
            )
            existing_round = cursor.fetchall()
            existing_round = existing_round[0] if existing_round else None
            # Remove tuple wrapper if there is only one round_id
            existing_round = existing_round[0] if existing_round else None

            # Insert or update the round
            if existing_round:
                cursor.execute(
                    f"UPDATE caselist SET {side}_team = ?, {side}_doc = ? WHERE round_id = ?",
                    (team_id, rnd['opensource'], existing_round)
                )
            else:
                cursor.execute(
                    f"INSERT INTO caselist (tournament_name, round_name, {side}_team, {side}_doc) VALUES (?, ?, ?, ?)",
                    (tournament, rnd['round'], team_id, rnd['opensource'])
                )
            cursor.close()
            conn.commit()
        else:
            continue


# commit changes
conn.commit()

Houston YaKh 1---NUSO 1 N Harvard DS ndtceda23/Houston/YaKh/Houston-YaKh-Neg-1---NUSO-Round-1.docx
tourn Northwestern
opponent Harvard DS 89043
Houston YaKh 1---NUSO 4 N Michigan HT ndtceda23/Houston/YaKh/Houston-YaKh-Neg-1---NUSO-Round-4.docx
tourn Northwestern
opponent Michigan HT 89195
Houston YaKh 1---NUSO 5 N Kansas WW ndtceda23/Houston/YaKh/Houston-YaKh-Neg-1---NUSO-Round-5.docx
tourn Northwestern
opponent Kansas WW 88836
Houston YaKh 2---Houston 2 N Kansas State SS ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Round-2.docx
tourn Houston
opponent Kansas State SS 89217
Houston YaKh 2---Houston 3 N Texas AD ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Round-3.docx
tourn Houston
opponent Texas AD 90362
Houston YaKh 2---Houston 5 N Texas CO ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Round-5.docx
tourn Houston
opponent Texas CO 90363
Houston YaKh 2---Houston Quarters N Trinity WP ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Quarters.docx
tourn Houston
oppo

In [37]:
bad_names = set()
cursor = conn.cursor()
cursor.execute("select tournament_name from caselist")
for row in cursor.fetchall():
    if row[0] not in tournaments:
        bad_names.add(row[0])

cursor.close()
print(bad_names)

{'OWEN L COON MEMORIAL DEBATES', 'Kentucky', 'D7', 'Contact Info', 'D5', 'Ziggy Online', 'University of Kentucky', 'JV Novice Nationals at West Virginia University', 'Crowe Warken Debates at USNA', 'Val Browning', 'Ziggy', 'GMU', 'Rutgers University Newark Brick City Round Robin', 'UKRR', 'D4', 'KU Swings', 'Liberty Debate Tournament', 'Owen L Coon Memorial Tournament', 'Crowe Warken Debates', 'D7777777', 'Indiana', 'uk', 'George Mason Debate Tournament', 'CSULB', 'KSU Swings', 'Shirley Classic', 'quals', 'navy', 'KU', 'CSU Long Beach', 'UMinn', 'sunflower swing pt 1', 'George-Mason University', 'Missouri State', 'University of Central Oklahoma', 'long beach', 'sunflower spring', 'Houston', 'NU', 'Val Browning Round Robin', 'sunflower swing pt1', 'University of Minnesota', 'Anti-trust', 'Crowe/Navy', 'HIT', 'New School', 'RRR', 'JV Novice Nats', 'shirley', 'National Debate Tournament', 'CSU', 'Liberty', 'KU Sunflower Swing Part 1', 'University of Houston College Tournament', 'West Poin

In [14]:
# if good name in bad name, replace bad name with good name
cursor = conn.cursor()
for bad_name in bad_names:
    for good_name in tournaments:
        if good_name.lower() in bad_name.lower() and "kansas" not in bad_name.lower():
            cursor.execute("update caselist set tournament_name = ? where tournament_name = ?", (good_name, bad_name))
            print("replacing", bad_name, "with", good_name)

# using the updated tournament names, combine rounds
cursor.execute("select * from caselist")
rounds = cursor.fetchall()
cursor.close()

cursor = conn.cursor()
for rnd in rounds:
    cursor.execute("select round_id from caselist where tournament_name = ? and round_name = ? and aff_team = ? and neg_team = ?", (rnd[1], rnd[2], rnd[3], rnd[4]))
    existing_round = cursor.fetchall()
    existing_round = existing_round[0] if existing_round else None
    existing_round = existing_round[0] if existing_round else None

    if existing_round:
        cursor.execute(
            "update caselist set aff_doc = aff_doc || ? where round_id = ?",
            (rnd[5], existing_round)
        )
        cursor.execute(
            "delete from caselist where round_id = ?",
            (rnd[0])
        )
    else:
        cursor.execute(
            "insert into caselist (tournament_name, round_name, aff_team, neg_team, aff_doc) values (?, ?, ?, ?, ?)",
            (rnd[1], rnd[2], rnd[3], rnd[4], rnd[5])
        )
# commit changes
cursor.close()
conn.commit()

replacing JW Patterson @ Kentucky with JW Patterson
replacing Texas Open with Texas
replacing JW Patterson Invitational with JW Patterson
replacing JW Patterson Debates hosted by UK with JW Patterson
replacing JW Patterson debates hosted by UK with JW Patterson
replacing texas with Texas
replacing Texas open with Texas
replacing Frank R. Shirley Wake Forest Debate Tournament with Wake
replacing Northwestern University Season Opener with Northwestern
replacing CEDA National Championships in Cali with CEDA
replacing ADA Nats with ADA
replacing NDT District VII Qualifier with NDT
replacing JW Patterson Kentucky with JW Patterson
replacing ADA Nationals at Indiana University with ADA
replacing harvard with Harvard
replacing Kathryn Klassic at the Beach 2024 with Kathryn
replacing ADA Fall Championship with ADA
replacing wake with Wake
replacing Kathryn Klassic at The Beach with Kathryn
replacing Kathryn Klassic with Kathryn
replacing Harvard (Online) with Harvard
replacing ADA Nationals wi

In [32]:
conn.commit()